In [1]:
#1. Import Libraries
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
#Load Raw Data
RAW_DATA_PATH = "../data/raw"

student_info = pd.read_csv(f"{RAW_DATA_PATH}/studentInfo.csv")
student_vle = pd.read_csv(f"{RAW_DATA_PATH}/studentVle.csv")
vle = pd.read_csv(f"{RAW_DATA_PATH}/vle.csv")
assessments = pd.read_csv(f"{RAW_DATA_PATH}/assessments.csv")
student_assessment = pd.read_csv(f"{RAW_DATA_PATH}/studentAssessment.csv")

In [3]:
#3. Clean studentInfo.csv & Create Churn Label
#3.1 Select Relevant Columns
student_info = student_info[ ['id_student', 'age_band', 'highest_education', 'final_result']]

In [4]:
#3.2 Create Churn Column
student_info['churn'] = student_info['final_result'].apply(lambda x: 1 if x == 'Withdrawn' else 0)
student_info['churn'].value_counts()

churn
0    22437
1    10156
Name: count, dtype: int64

In [5]:
#4. Clean & Engineer Engagement Features (studentVle.csv) 
#4.1 Basic Cleaning
student_vle = student_vle[['id_student', 'id_site', 'date', 'sum_click']]

In [7]:
#4.2 Student-Level Engagement Aggregation
vle_features = student_vle.groupby('id_student').agg(
    total_clicks=('sum_click', 'sum'),
    avg_clicks=('sum_click', 'mean'),
    max_clicks=('sum_click', 'max'),
    active_days=('date', 'nunique'),
    first_active_day=('date', 'min'),
    last_active_day=('date', 'max')
).reset_index()

In [8]:
#4.3 Inactivity Feature (Early Churn Signal)
max_day = student_vle['date'].max()

vle_features['inactive_14_days'] = (
    vle_features['last_active_day'] < (max_day - 14)
).astype(int)

In [9]:
#5. Assessment Behavior Features
#5.1 Merge Submissions with Deadlines
assessment_merged = student_assessment.merge(
    assessments[['id_assessment', 'date']],
    on='id_assessment',
    how='left',
    suffixes=('_submitted', '_deadline')
)

In [11]:
#5.2 Create Student-Level Assessment Metrics
assessment_features = assessment_merged.groupby('id_student').agg(
    avg_score=('score', 'mean'),
    max_score=('score', 'max'),
    min_score=('score', 'min'),
    missed_assessments=('score', lambda x: x.isna().sum()),
    submissions=('score', 'count')
).reset_index()

In [12]:
# 6. Resource-Type Engagement Features (ADVANCED)
#6.1 Merge VLE with Resource Type
vle_full = student_vle.merge(
    vle[['id_site', 'activity_type']],
    on='id_site',
    how='left'
)

In [13]:
#6.2 Pivot Engagement by Resource Type
resource_features = vle_full.pivot_table(
    index='id_student',
    columns='activity_type',
    values='sum_click',
    aggfunc='sum',
    fill_value=0
).reset_index()

In [14]:
#7. Merge All Features into Final Dataset
final_df = (
    student_info
    .merge(vle_features, on='id_student', how='left')
    .merge(assessment_features, on='id_student', how='left')
    .merge(resource_features, on='id_student', how='left')
)

In [15]:
#8. Handle Missing Values
final_df.fillna(0, inplace=True)

In [16]:
#9. Final Dataset Check
final_df.shape
final_df.head()
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id_student          32593 non-null  int64  
 1   age_band            32593 non-null  object 
 2   highest_education   32593 non-null  object 
 3   final_result        32593 non-null  object 
 4   churn               32593 non-null  int64  
 5   total_clicks        32593 non-null  float64
 6   avg_clicks          32593 non-null  float64
 7   max_clicks          32593 non-null  float64
 8   active_days         32593 non-null  float64
 9   first_active_day    32593 non-null  float64
 10  last_active_day     32593 non-null  float64
 11  inactive_14_days    32593 non-null  float64
 12  avg_score           32593 non-null  float64
 13  max_score           32593 non-null  float64
 14  min_score           32593 non-null  float64
 15  missed_assessments  32593 non-null  float64
 16  subm

In [22]:
import os
os.getcwd()

'C:\\Users\\tirth\\EdTech-Student-Churn\\notebooks'

In [23]:
os.makedirs("../data/processed", exist_ok=True)

final_df.to_csv(
    "../data/processed/edtech_churn_final.csv",
    index=False
)

In [25]:
os.listdir("../data/processed")

['edtech_churn_final.csv']

11. Summary (Markdown Cell – VERY IMPORTANT)

Key Outcomes:

Created churn label using final results

Engineered engagement-based early churn indicators

Aggregated assessment performance features

Built a unified student-level dataset for analytics

END OF NOTEBOOK 02
What we Have Achieved:
✔ Cleaned raw datasets
✔ Created advanced churn features
✔ Built a modeling-ready dataset
✔ Followed industry workflow
